In [1]:
import numpy as np
from copy import deepcopy

In [2]:
class MinCostFlowSolver:
    def __init__(self, costs_graph, costs, initial_plan, initial_basis):
        self.costs_graph = deepcopy(costs_graph)
        self.costs = deepcopy(costs)
        self.initial_basis = deepcopy(initial_basis)
        self.initial_plan = deepcopy(initial_plan)
        
        
    def get_cycle_path(self, graph):
        n = len(graph)
        visited = [False]*(n)
        path = []
          
        for i in range(n):    
            if visited[i] == False: 
                path = []
                if self.get_cycle_path_util(graph, i, visited, -1, path) == True:
                    result =  [i for i in reversed(path)]
                    return self.get_edges_from_cycle(result)
          
        return self.get_edges_from_cycle(path)

    def get_cycle_path_util(self, graph, v, visited, parent, path):
        visited[v] = True
    
        for i in range(len(graph[v])): 
            if graph[v][i] > 0:
                if visited[i] == False:
                    if self.get_cycle_path_util(graph, i, visited, v, path):
                        if len(path) == len(set(path)):
                            path.append(v)
                        
                        return True
                elif parent != i and parent != -1:
                    path.append(i)
                    path.append(v)
                    return True
          
        return False

    def get_edges_from_cycle(self, cycle):
        edges = []
        for i in range(1, len(cycle)):
            edges.append((cycle[i - 1], cycle[i]))
        
        return edges
    
    def get_direction(self, edge):
        if self.costs_graph[edge[0]][edge[1]] != None:
            return 'ASC'
        elif self.costs_graph[edge[1]][edge[0]] != None:
            return 'DESC'
        
    def index_or_minus(self, _list, item):
        try:
            index = _list.index(item)
        except ValueError:
            index = -1
            
        return index
        
        
    def solve(self):
        current_basis = self.initial_basis
        current_plan = self.initial_plan
        
        
        while True:
            
            # 1
            u = [None] * len(self.costs_graph)
            u[0] = 0
            while any(v is None for v in u):
                for start, end in current_basis:
                    if u[start] == None and u[end] == None:
                        continue
                    else:
                        if u[start] == None and u[end] != None:
                            u[start] = u[end] + self.costs_graph[start][end]
                        elif u[end] == None and u[start] != None:
                            u[end] = u[start] - self.costs_graph[start][end]

            # 2
            non_basis_edges = []
            for i in range(len(self.costs_graph[0])):
                for j in range(len(self.costs_graph)):
                    if self.costs_graph[i][j] != None and (i, j) not in current_basis:
                        non_basis_edges.append((i, j))
            

            delta = []
            for start, end in non_basis_edges:
                delta.append(u[start] - u[end] - self.costs_graph[start][end])
            

            # 3
            edge_0 = None
            for index, item in enumerate(delta):
                if item > 0:
                    edge_0 = non_basis_edges[index]
                    break
            if edge_0 == None: 
                return current_plan

            # 4
            current_basis.append(edge_0)

            n = len(self.costs_graph)
            undirected_graph = [([0]*n) for i in range(n)]

            for start, end in current_basis:
                undirected_graph[start][end] = 1
                undirected_graph[end][start] = 1

            cycle = self.get_cycle_path(undirected_graph)

            # 5
            index_0 = self.index_or_minus(cycle, (edge_0[0], edge_0[1]))
            if index_0 == -1:
                index_0 = self.index_or_minus(cycle, (edge_0[1], edge_0[0]))

            m = len(cycle) - index_0
            cycle = cycle[index_0:] + cycle[:-m]

            pos = []
            neg = []

            _min = np.inf
            min_edge = None

            direction = self.get_direction(edge_0)

            for edge in cycle:
                if self.get_direction(edge) != direction:
                    neg.append(edge)
                    if current_plan[edge[0]][edge[1]] != None:
                        _min = min(_min, current_plan[edge[0]][edge[1]])
                        min_edge = (edge[0], edge[1])
                    else:
                        _min = min(_min, current_plan[edge[1]][edge[0]])  
                        min_edge = (edge[1], edge[0])
                else:
                    pos.append(edge)

            # 6
            for edge in cycle:
                if edge in pos:
                    if current_plan[edge[0]][edge[1]] != None:
                        current_plan[edge[0]][edge[1]] += _min
                    else:
                        current_plan[edge[1]][edge[0]] += _min
                elif edge in neg:
                    if current_plan[edge[0]][edge[1]] != None:
                        current_plan[edge[0]][edge[1]] -= _min
                    else:
                        current_plan[edge[1]][edge[0]] -= _min

            current_basis.remove(min_edge)
            
        return 'No solution'

In [3]:
initial_basis = [
    (0, 1),
    (2, 1),
    (5, 2),
    (2, 3),
    (4, 3)
]
costs_graph = [
    [None, 1, None, None, None, None],
    [None, None, None, None, None, 3],
    [None, 3, None, 5, None, None],
    [None, None, None, None, None, None],
    [None, None, 4, 1, None, None],
    [-2, None, 3, None, 4, None],
]
initial_plan = [
    [None, 1, None, None, None, None],
    [None, None, None, None, None, 0],
    [None, 3, None, 1, None, None],
    [None, None, None, None, None, None],
    [None, None, 0, 5, None, None],
    [0, None, 9, None, 0, None],
]
costs = [1, -4, -5, -6, 5, 9]

In [4]:
solver = MinCostFlowSolver(costs_graph, costs, initial_plan, initial_basis)
print('Result:\n')
res = solver.solve()

print('     ', end='')
for i in range(len(res[0])):
    print(i + 1, end=' ')
print()

for i in range(len(res[0])):
    print(i + 1, ' | ', end='')
    for j in range(len(res)):
        if res[i][j] == None:
            print('-', end=' ')
        else:
            print(res[i][j], end=' ')
    print()

Result:

     1 2 3 4 5 6 
1  | - 4 - - - - 
2  | - - - - - 0 
3  | - 0 - 1 - - 
4  | - - - - - - 
5  | - - 0 5 - - 
6  | 3 - 6 - 0 - 
